In [2]:
reset -fs

In [4]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sns.set(context='notebook', style='whitegrid', font_scale=1.2)

In [5]:
nba = pd.read_csv('/Users/kyleturner/Regression-Project/NBA Player Table')

In [6]:
nba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1315 non-null   int64  
 1   #           1315 non-null   int64  
 2   Player      1315 non-null   object 
 3   GP          1315 non-null   int64  
 4   MIN         1315 non-null   int64  
 5   PTS         1315 non-null   int64  
 6   FGM         1315 non-null   int64  
 7   FGA         1315 non-null   int64  
 8   FG%         1315 non-null   float64
 9   3PM         1315 non-null   object 
 10  3PA         1315 non-null   object 
 11  3P%         1315 non-null   object 
 12  FTM         1315 non-null   int64  
 13  FTA         1315 non-null   int64  
 14  FT%         1315 non-null   float64
 15  OREB        1315 non-null   object 
 16  DREB        1315 non-null   object 
 17  REB         1315 non-null   int64  
 18  AST         1315 non-null   int64  
 19  STL         1315 non-null  

In [7]:
nba

,Unnamed: 0,#,Player,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,eFG%,TS%
0,0,1,Kareem Abdul-Jabbar,1560,57446,38387,15837,28307,55.9,1,18,5.6,6712,9304,72.1,2975,9394,17440,5660,1160,3189,2527,55.9,59.2
1,1,2,Karl Malone,1476,54852,36928,13528,26210,51.6,85,310,27.4,9787,13188,74.2,3562,11406,14968,5248,2085,1145,4524,51.8,57.7
2,2,3,LeBron James,1343,51265,36319,13258,26291,50.4,2074,5999,34.6,7729,10532,73.4,1574,8428,10002,9908,2116,1019,4694,54.4,58.7
3,3,4,Kobe Bryant,1346,48643,33643,11719,26200,44.7,1827,5546,32.9,8378,10011,83.7,1499,5548,7047,6306,1944,640,4010,48.2,55.0
4,4,5,Michael Jordan,1072,41010,32292,12192,24537,49.7,581,1778,32.7,7327,8772,83.5,1668,5004,6672,5633,2514,893,2924,50.9,56.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,1310,1311,Jim McIlvaine,401,5931,1072,423,948,44.6,1,10,10.0,225,408,55.1,479,764,1243,105,136,691,231,44.7,47.5
1311,1311,1312,Joel Anthony,490,7056,1064,398,776,51.3,0,0,0.0,268,405,66.2,562,791,1353,99,127,531,227,51.3,55.8
1312,1312,1313,Greg Dreiling,474,4239,1014,375,803,46.7,3,9,33.3,261,402,64.9,287,731,1018,186,82,137,250,46.9,51.7
1313,1313,1314,Mark Madsen,453,5335,975,381,834,45.7,1,16,6.3,212,402,52.7,559,598,1157,181,116,102,206,45.7,48.2


In [8]:
nba.columns = [column.strip().lower() for column in nba.columns] #reformat

In [13]:
nba

,unnamed: 0,#,player,gp,min,pts,fgm,fga,fg%,3pm,3pa,3p%,ftm,fta,ft%,oreb,dreb,reb,ast,stl,blk,tov,efg%,ts%
0,0,1,Kareem Abdul-Jabbar,1560,57446,38387,15837,28307,55.9,1,18,5.6,6712,9304,72.1,2975,9394,17440,5660,1160,3189,2527,55.9,59.2
1,1,2,Karl Malone,1476,54852,36928,13528,26210,51.6,85,310,27.4,9787,13188,74.2,3562,11406,14968,5248,2085,1145,4524,51.8,57.7
2,2,3,LeBron James,1343,51265,36319,13258,26291,50.4,2074,5999,34.6,7729,10532,73.4,1574,8428,10002,9908,2116,1019,4694,54.4,58.7
3,3,4,Kobe Bryant,1346,48643,33643,11719,26200,44.7,1827,5546,32.9,8378,10011,83.7,1499,5548,7047,6306,1944,640,4010,48.2,55.0
4,4,5,Michael Jordan,1072,41010,32292,12192,24537,49.7,581,1778,32.7,7327,8772,83.5,1668,5004,6672,5633,2514,893,2924,50.9,56.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,1310,1311,Jim McIlvaine,401,5931,1072,423,948,44.6,1,10,10.0,225,408,55.1,479,764,1243,105,136,691,231,44.7,47.5
1311,1311,1312,Joel Anthony,490,7056,1064,398,776,51.3,0,0,0.0,268,405,66.2,562,791,1353,99,127,531,227,51.3,55.8
1312,1312,1313,Greg Dreiling,474,4239,1014,375,803,46.7,3,9,33.3,261,402,64.9,287,731,1018,186,82,137,250,46.9,51.7
1313,1313,1314,Mark Madsen,453,5335,975,381,834,45.7,1,16,6.3,212,402,52.7,559,598,1157,181,116,102,206,45.7,48.2


In [17]:
# Drop unnecessary columns
df = nba.drop(['#', 'fg%', '3p%','ft%','reb', 'ast','stl', 'blk','unnamed: 0'], axis=1)

In [18]:
df.columns

Index(['player', 'gp', 'min', 'pts', 'fgm', 'fga', '3pm', '3pa', 'ftm', 'fta',
       'oreb', 'dreb', 'tov', 'efg%', 'ts%'],
      dtype='object')

In [19]:
df

,player,gp,min,pts,fgm,fga,3pm,3pa,ftm,fta,oreb,dreb,tov,efg%,ts%
0,Kareem Abdul-Jabbar,1560,57446,38387,15837,28307,1,18,6712,9304,2975,9394,2527,55.9,59.2
1,Karl Malone,1476,54852,36928,13528,26210,85,310,9787,13188,3562,11406,4524,51.8,57.7
2,LeBron James,1343,51265,36319,13258,26291,2074,5999,7729,10532,1574,8428,4694,54.4,58.7
3,Kobe Bryant,1346,48643,33643,11719,26200,1827,5546,8378,10011,1499,5548,4010,48.2,55.0
4,Michael Jordan,1072,41010,32292,12192,24537,581,1778,7327,8772,1668,5004,2924,50.9,56.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,Jim McIlvaine,401,5931,1072,423,948,1,10,225,408,479,764,231,44.7,47.5
1311,Joel Anthony,490,7056,1064,398,776,0,0,268,405,562,791,227,51.3,55.8
1312,Greg Dreiling,474,4239,1014,375,803,3,9,261,402,287,731,250,46.9,51.7
1313,Mark Madsen,453,5335,975,381,834,1,16,212,402,559,598,206,45.7,48.2


In [22]:
df.isnull().sum()

player    0
gp        0
min       0
pts       0
fgm       0
fga       0
3pm       0
3pa       0
ftm       0
fta       0
oreb      0
dreb      0
tov       0
efg%      0
ts%       0
dtype: int64

In [24]:
df.head(15)

,player,gp,min,pts,fgm,fga,3pm,3pa,ftm,fta,oreb,dreb,tov,efg%,ts%
0,Kareem Abdul-Jabbar,1560,57446,38387,15837,28307,1,18,6712,9304,2975,9394,2527,55.9,59.2
1,Karl Malone,1476,54852,36928,13528,26210,85,310,9787,13188,3562,11406,4524,51.8,57.7
2,LeBron James,1343,51265,36319,13258,26291,2074,5999,7729,10532,1574,8428,4694,54.4,58.7
3,Kobe Bryant,1346,48643,33643,11719,26200,1827,5546,8378,10011,1499,5548,4010,48.2,55.0
4,Michael Jordan,1072,41010,32292,12192,24537,581,1778,7327,8772,1668,5004,2924,50.9,56.9
5,Dirk Nowitzki,1522,51367,31560,11169,23734,1982,5210,7240,8239,1468,10021,2494,51.2,57.7
6,Wilt Chamberlain,1045,47859,31419,12681,23497,-,-,6057,11862,-,-,-,54.0,54.7
7,Shaquille O'Neal,1207,41917,28596,11330,19457,1,22,5935,11252,4209,8890,3310,58.2,58.6
8,Carmelo Anthony,1233,42857,27925,9990,22363,1678,4728,6267,7703,1958,5739,3027,48.4,54.2
9,Moses Malone,1329,45071,27409,9435,19225,8,80,8531,11090,6731,9481,3804,49.1,56.9
